# Barcode Matrix
Here we attempt to define a graphical representation of fixed-length genetic barcodes.

The aim is to create a graphical representation that is *complete* (i.e. includes all barcodes in its domain).

## 1. Properties of Barcode Sets

In order to define a graphical representation of $k$-length genetic barcodes, let us consider the following properties of the set $\text{B}_k$ of such barcodes:
1. Each barcode $b$ in $\text{B}_k$ is a sequence of *quaternary symbols*:
    \begin{equation}
    b = (b_1, b_2, ..., b_k)
    \end{equation}
2. Each symbol in a barcode can take on the following values:
    \begin{equation}
    b_i \in \{A, C, T, G\} \quad (1 \leq i \leq k)
    \end{equation}
3. The cardinality of the set, or the theoretical diversity of barcodes, is therefore:
    \begin{equation}
    |\text{B}_k| = 4^k
    \end{equation}

## 2. Counting Barcodes Recursively

One useful starting point for conceiving a complete graphical representation in $\text{B}_k$ is to consider the relationship between barcode set cardinalities.

In particular, if we are counting barcodes in $\text{B}_k$, we can do so by recursing on the size of $\text{B}$ from $k\to1$.
\begin{align}
|\text{B}_k| = 4|\text{B}_{k-1}| \\
|\text{B}_1| = 4
\end{align}

By this recurrence, we can see that as we move along the sequence $b$, number of possible $b$ expands by powers of 4.

Since the symbols of $b$ can be read arbitrarily in either direction, this recursion counts the number of possible barcodes accumulating from $b_1$ to $b_k$.

## 3. Representing Barcode Sets as a Recursive Block Matrix

The required graphical representation $\text{G}_k$ for a barcode set $B_k$ must be able to represent every element $b$ in $\text{B}_k$.

To do this, we will exploit the unique topology of a **square tiling** (Schlafli symbol $\{4,4\}$) to construct a recursive block matrix:
\begin{align}
\text{G}_k =
\begin{bmatrix}
\text{G}_{k-1} & \text{G}_{k-1} \\
\text{G}_{k-1} & \text{G}_{k-1}
\end{bmatrix} \\
\text{G}_1 =
\begin{bmatrix}
A & C \\
T & G
\end{bmatrix}
\end{align}

The above recursive definition for $G_k$ leads to a graph with the following desirable properties:
1. $G_k$ is a square matrix with dimensions $2^k \times 2^k$
2. The number of entries in $G_k$ is $4^k = |\text{B}_k|$

Hence, $G_k$ is a complete matrix representation of $B_k$.




## 4. Indexing Barcodes Recursively

The entries of the so-called *"barcode matrix"* $\text{G}_k$ can be indexed recursively.

For any given $b$, the barcode can be accessed by performing a recursive walk in $\text{G}_k$:
\begin{align}
b = (b_1, b_2, ..., b_k) \\
G_k: b \mapsto g_1^k \\
g_1^k::= (\text{G}_k \to \text{G}_{k-1} \to \text{G}_{k-2} \to ... \to \text{G}_1) \\
\end{align}

We use the arbitrary linear ordering of elements in $b$ to allow indexing by descent from $k \to 1$ in $\text{G}_k$.

## 5. The Structure of the Matrix is Determined by the Base Case

As a final note, the structure of the barcode matrix $\text{G}_k$ is deterministically set by the order of entries in the base case $\text{G}_1$.
\begin{equation}
\text{G}_1 =
\begin{bmatrix}
g_{11} & g_{12} \\
g_{21} & g_{22}
\end{bmatrix}
\end{equation}
In this instance, we have defined the order $\text{G}_1 = (g_{11}, g_{12}, g_{21}, g_{22}) = (A, C, T, G)$.


## 6. Determining the Indexes of a Barcode by Descent

Taken together, the above properties of $\text{G}_k$ allow us to generate an algorithm for finding the indexes of any given barcode $b$ by descent on the limits of the indexes.

**Algorithm 1** Indexing by Descent

**Input:** $b, G_1$
**Output:** $x, y$

1. **function** $\textbf{index}(b, G_1)$
2. &ensp;$k\leftarrow\text{dim}(b)$
3. &ensp;$x\leftarrow[1, 2^k]$
4. &ensp;$y\leftarrow[1, 2^k]\cdot$
5. &ensp;$\textbf{for }i\leftarrow 1\text{ to }k\textbf{ do}$
6. &emsp;$\textbf{if }b_i \in [g_{11}, g_{12}]\textbf{ then}$
7. &emsp;&ensp;$x\leftarrow \left[ x_1, \frac{x_2-x_1+1}{2}\right]$
8. &emsp;$\textbf{else}$
9. &emsp;&ensp;$x\leftarrow \left[ \frac{x_2-x_1+1}{2}+1, x_2 \right]$
10. &emsp;$\textbf{if }b_i \in [g_{11}, g_{21}]\textbf{ then}$
11. &emsp;&ensp;$y\leftarrow \left[ y_1, \frac{y_2-y_1+1}{2}\right]$
12. &emsp;$\textbf{else}$
13. &emsp;&ensp;$y\leftarrow \left[ \frac{y_2-y_1+1}{2}+1, y_2 \right]$
14. &ensp;$\textbf{end for}$
15. &ensp;$\textbf{return }x, y$
16. $\textbf{end function}$

Since $G_k$ has dimensions $2^k \times 2^k$ and the algorithm halves the interval around the indexes at each step, within $k$ steps the limits will converge and we get $x, y$ for $b$.

## 7. The Matrix Can Be Indexed in Linear Time

If it takes $k$ steps to index each barcode, then the computational complexity of the above algorithm for $|B_k| = n$ is:
\begin{equation}
\Theta(n) = n
\end{equation}



## 8. Implementation of the Indexing Algorithm

In [12]:
import numpy as np

def get_indexes(barcode: str, base: np.array) -> list[int]:
    """
    Get indexes of barcode by descent using the base case matrix.
    """
    # Length of barcode
    k = len(barcode)

    # Initiate indexes
    x, y = [0, 2**k], [0, 2**k]

    # Find indexes by limit descent
    for i in range(k):
        j = np.where(base == barcode[i])
        x = [x[0], (x[1]-x[0]+1)/2] if j[0] == 0 else [(x[1]-x[0]+1)/2 + 1, x[1]]
        y = [y[0], (y[1]-y[0]+1)/2] if j[1] == 0 else [(y[1]-y[0]+1)/2 + 1, y[1]]

    # Return indexes
    return [x, y]
